In [ ]:
import requests
import re
import os
import time

In [ ]:
from collections import OrderedDict, defaultdict
from pprint import saferepr

# 把每行的 (XX, YY) 作为 key, value 对加入到 dict 中去
def readTuplesToDict(sFileName, sEncoding = 'utf-8', sType = 'dict'):
    ls = []
    try:
        with open(sFileName, 'r', encoding = sEncoding, errors = 'ignore') as fl:
            for s in fl.readlines():
                s = s.strip()
                if s == '': continue
                if s[0] == '#': continue    # Ignore comments
                t = ast.literal_eval(s)
                ls.append(t)
            #end for
        #end with
    except Exception as e:
        print(e)
    #end try

    if sType == 'OrderedDict':
        return OrderedDict(ls)
    elif sType == 'ListDict':
        return defaultdict(list, ls)
    elif sType == 'DictDict':
        return defaultdict(dict, ls)
    else:
        return dict(ls)
#def

def writeTupleDict(d, sFileName, sEncoding = 'utf-8', bReverse = False):
    ls = sorted(d.items(), key = lambda t:t[0], reverse = bReverse)
    with open(sFileName, 'w', encoding = sEncoding, errors = 'ignore') as fl:
        for item in ls:
            #sLine = repr(item)
            sLine = saferepr(item)
            print(sLine, file = fl)
        #end for
    #end with
#def

In [ ]:
rxEng = re.compile(r'[A-Za-z0-9\s]+')
def genNameForFile(s):
    m = rxEng.search(s)
    if m:
        sEng = m.group(0).replace(' ', '')
        if sEng != '':
            return sEng
    return s.replace(' ', '')

In [ ]:
def getIdFromUrl(sUrl):
    sUrl = sUrl.rstrip('/')
    return sUrl[sUrl.rfind('/') + 1 : ]

In [ ]:
sDownDir = '../tempdown'

In [ ]:
rxName = re.compile(r'<h1>(.+?)</h1>')

rxGender = re.compile(r'性别.{1,10}:(.+?)<', re.DOTALL)

rxPplUrl = re.compile(r'"(.{1,9}movie\.douban\.com/celebrity/[^/]+/)"')

rxPicList = re.compile(r'class="cover".+?a href="(.+?)".+?img src="(.+?)".+?class="prop">(.+?)</div>', re.DOTALL)

rxPicCnt = re.compile(r'span class="count">[^\d]{1,5}(\d+)[^\d]{1,5}</span', re.DOTALL)

In [ ]:
dVisited = readTuplesToDict('douban.txt', sType='DictDict')

In [205]:
if '_last' in dVisited:
    sLastPplId = dVisited['_last']
    sLastPplUrl = dVisited[sLastPplId]['url']
    sLastPplName = dVisited[sLastPplId]['long_name']
else:
    sLastPplId = '1049504'       # seed
    sLastPplUrl = 'https://movie.douban.com/celebrity/{}/'.format(sLastPplId)
    sLastPplName = ''

print('Last visited person: {} {}'.format(sLastPplName, sLastPplUrl))

Last visited person: 罗丝·麦高恩 Rose McGowan https://movie.douban.com/celebrity/1049504/


In [206]:
lsPplUrls = [sLastPplUrl]

#while True:
for i in range(20):
    if i >= len(lsPplUrls):
        break
    
    bDown = True
    sPplUrl = lsPplUrls[i]
    sPplId = getIdFromUrl(sPplUrl)
    
    print('Visiting {}'.format(sPplUrl))
    rspCeleb = requests.get(sPplUrl)
    
    # Add related people to list
    iPtnr = rspCeleb.text.find('div id="partners"')
    lsPtnrUrls = rxPplUrl.findall(rspCeleb.text[iPtnr:])
    for sPtnUrl in lsPtnrUrls:
        sPtnId = getIdFromUrl(sPtnUrl)
        if (sPtnId not in dVisited) and (sPtnUrl not in lsPplUrls):
            lsPplUrls.append(sPtnUrl)
    
    m = rxGender.search(rspCeleb.text)
    if m:
        sGender = m.group(1).strip()
        bDown = bDown and ('女' in sGender)
    
    if not bDown:
        continue

    m = rxName.search(rspCeleb.text)
    sLongName = m.group(1)
    sName = genNameForFile(sLongName)
    
    dVisited['_last'] = sPplId
    if sPplId in dVisited:
        iLastPicId = dVisited[sPplId]['last_pic_id']
    else:
        iLastPicId = -1
        dPpl = {}
        dPpl['url'] = sPplUrl
        dPpl['long_name'] = sLongName
        dPpl['last_pic_id'] = iLastPicId
        dVisited[sPplId] = dPpl

    # Visit the gallery page
    sGalrUrl = '{}photos/?sortby=time'.format(sPplUrl)
    sRef = sPplUrl
    nTotPics = 0
    iBgnPic = 0
    bRecd = False
    while True:   # loop through gallery pages
        rspGllr = requests.get(sGalrUrl, headers={'Referer':sRef})

        if nTotPics == 0:    # Get total count of pics
            m = rxPicCnt.search(rspGllr.text)
            if m:
                nTotPics = int(m.group(1))
                print('{} has {} photos'.format(sLongName, nTotPics))
            else:
                print('{} has no photos'.format(sLongName, nTotPics))
                break

        lsPics = rxPicList.findall(rspGllr.text)

        for sPicPage, sPicUrl, sSize in lsPics:
            sSize = sSize.strip()
            sWidth, sHeight = sSize.split('x')
            nWidth = int(sWidth)
            nHeight = int(sHeight)
            if not ((nWidth >= 600 and nHeight >= 900) or (nWidth >= 1200 and nHeight >= 700)):
                #print('{} is too small'.format(sSize))
                continue
                
            iPicId = int(getIdFromUrl(sPicPage))
            if iPicId <= iLastPicId:
                break
            if not bRecd:
                dVisited[sPplId]['last_pic_id'] = iPicId
                bRecd = True
            sPicUrl = sPicUrl.replace('/m/', '/raw/')    # replace 'm' with 'raw' to get the raw pic
            #print('{}\t{}\t{}'.format(sSize, iPicId, sPicUrl))
            rspPic = requests.get(sPicUrl, headers={'Referer':sPicPage})
            sFilePath = '{}/{}_{}_{}'.format(sDownDir, sName, sPplId, getIdFromUrl(sPicUrl))
            print('Downloading {}...'.format(sFilePath))
            with open(sFilePath, 'wb') as fl:
                fl.write(rspPic.content)
            time.sleep(1)
        #end for

        if iPicId <= iLastPicId:
            break

        iBgnPic = iBgnPic + len(lsPics)
        if iBgnPic >= nTotPics:
            break

        sRef = sGalrUrl
        sGalrUrl = '{}photos/?sortby=time&start={}'.format(sPplUrl, iBgnPic)

    #end looping gallery pages
    
    writeTupleDict(dVisited, 'douban.txt')
    
#end for looping celeb url list

Visiting https://movie.douban.com/celebrity/1049504/
罗丝·麦高恩 Rose McGowan has 506 photos
Visiting https://movie.douban.com/celebrity/1054444/
Visiting https://movie.douban.com/celebrity/1004568/
Visiting https://movie.douban.com/celebrity/1010509/
Visiting https://movie.douban.com/celebrity/1019016/
Visiting https://movie.douban.com/celebrity/1054408/
Visiting https://movie.douban.com/celebrity/1031221/
乌玛·瑟曼 Uma Thurman has 436 photos


Visiting https://movie.douban.com/celebrity/1027816/
Visiting https://movie.douban.com/celebrity/1002692/
Visiting https://movie.douban.com/celebrity/1041005/
Visiting https://movie.douban.com/celebrity/1054443/
Visiting https://movie.douban.com/celebrity/1054453/
斯嘉丽·约翰逊 Scarlett Johansson has 2680 photos


Visiting https://movie.douban.com/celebrity/1054517/
Visiting https://movie.douban.com/celebrity/1054527/
Visiting https://movie.douban.com/celebrity/1054401/
Visiting https://movie.douban.com/celebrity/1022570/
Visiting https://movie.douban.com/celebrity/1035643/
Visiting https://movie.douban.com/celebrity/1044721/
Visiting https://movie.douban.com/celebrity/1022582/
Visiting https://movie.douban.com/celebrity/1047986/
萨尔玛·海耶克 Salma Hayek has 260 photos


In [ ]:
iLastPicId

In [207]:
dVisited

defaultdict(dict,
            {'1002691': {'last_pic_id': 2459881811,
              'long_name': '玛丽·谢尔顿 Marley Shelton',
              'url': 'https://movie.douban.com/celebrity/1002691/'},
             '1004668': {'last_pic_id': -1,
              'long_name': '凯茜·纳基麦 Kathy Najimy',
              'url': 'https://movie.douban.com/celebrity/1004668/'},
             '1012520': {'last_pic_id': 2409352505,
              'long_name': '艾伦·佩吉 Ellen Page',
              'url': 'https://movie.douban.com/celebrity/1012520/'},
             '1022588': {'last_pic_id': 2495461019,
              'long_name': '克里斯汀·韦格 Kristen Wiig',
              'url': 'https://movie.douban.com/celebrity/1022588/'},
             '1022590': {'last_pic_id': 2391876790,
              'long_name': '杰米·普莱斯利 Jaime Pressly',
              'url': 'https://movie.douban.com/celebrity/1022590/'},
             '1022623': {'last_pic_id': 2497984001,
              'long_name': '吉莲·安德森 Gillian Anderson',
              'url': 'https

In [204]:
#dVisited['1049504']['last_pic_id'] = 2445682081
dVisited['_last'] = '1049504'

In [199]:
lsPplUrls

['https://movie.douban.com/celebrity/1004668/',
 'https://movie.douban.com/celebrity/1035650/',
 'https://movie.douban.com/celebrity/1041136/',
 'https://movie.douban.com/celebrity/1077033/',
 'https://movie.douban.com/celebrity/1286024/',
 'https://movie.douban.com/celebrity/1027847/']

In [ ]:
lsPtnrUrls

In [ ]:
set(rxPplUrl.findall(rspCeleb.text[iPtnr:]))

In [ ]:
for sPicPage, sPicUrl, sSize in lsPics:
    sSize = sSize.strip()
    sWidth, sHeight = sSize.split('x')
    nWidth = int(sWidth)
    nHeight = int(sHeight)
    if not ((nWidth >= 600 and nHeight >= 900) or (nWidth >= 1200 and nHeight >= 700)):
        print('{} is too small'.format(sSize))
        continue
    nPicId = int(getIdFromUrl(sPicPage))
    sPicUrl = sPicUrl.replace('/m/', '/raw/')    # replace 'm' with 'raw' to get the raw pic
    print('{}\t{}\t{}'.format(sSize, nPicId, sPicUrl))

In [ ]:
print(rsp.text)

In [ ]:
sUrl = 'https://img3.doubanio.com/view/photo/raw/public/p2161005046.jpg'

In [ ]:
dHeader = {}
dHeader['Referer'] = 'https://movie.douban.com/celebrity/1049504/photo/2161005046/'
#dHeader['Cookie'] = 'll="108296"; bid=mVp3radEseU; _vwo_uuid_v2=4952852E45FA9BB3C6EFDDD3F0C022BE|01d5e5e89c115b916a2e917600df90b8; ap=1; dbcl2="2297010:gA6qmptTYSg"; ck=eRvo; __utma=30149280.554498440.1500881853.1507776718.1507783081.9; __utmb=30149280.0.10.1507783081; __utmc=30149280; __utmz=30149280.1507695628.4.3.utmcsr=bing|utmccn=(organic)|utmcmd=organic|utmctr=%E5%B0%BC%E6%B3%8A%E5%B0%94%20%E6%B3%B0%E7%B1%B3%E5%B0%94%20dance%20bar; push_noty_num=0; push_doumail_num=0'


In [ ]:
rsp = requests.get(sUrl, headers=dHeader)

In [ ]:
rsp

In [ ]:
len(rsp.content)

In [ ]:
fl = open('p2161005046.jpg', 'wb')

In [ ]:
fl.write(rsp.content)

In [ ]:
fl.close()